## How much loss do we have in the process?

In this notebook we see how many X17-A pass through each step in the process (from step3 - taking pdf subset, to step 8 - creating structured asset and liable database)

In [ ]:
from FocusReportSlicing import selectPages, extractSubset, brokerFilter
from ExtractBrokerDealers import dealerData

import time
import boto3
from sagemaker.session import Session
import sys
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json

company_email = 'mathias.andler@ny.frb.org'
s3_pointer = boto3.client('s3')
s3_session = Session()

s3_bucket = "x17-a5-mathias-version-nit"
temp_folder ='temp/'

In [2]:
from GLOBAL import GlobVars
export_pdf = GlobVars.temp_folder_pdf_slice
export_png = GlobVars.temp_folder_png_slice
input_raw = GlobVars.input_folder_raw
input_pdf = temp_folder + 'X-17A-5-PDF-SUBSETS/'
out_folder_raw_pdf = temp_folder + 'X-17A-5-PDF-RAW/'

In [ ]:
temp_folder
out_folder_clean_pdf, 
out_folder_clean_png
out_folder_split_pdf
out_folder_split_png
'input/X-17A-5/'

## After step 2: total PDFs downloaded

In [8]:
x17 = s3_session.list_s3_files(s3_bucket,'input/X-17A-5/')

In [33]:
x17

['input/X-17A-5/',
 'input/X-17A-5/1000146-2002-03-13.pdf',
 'input/X-17A-5/1000146-2003-03-03.pdf',
 'input/X-17A-5/1000146-2004-03-08.pdf',
 'input/X-17A-5/1000147-2002-02-28.pdf',
 'input/X-17A-5/1000147-2003-02-28.pdf',
 'input/X-17A-5/1000147-2004-02-25.pdf',
 'input/X-17A-5/1000147-2004-03-29.pdf',
 'input/X-17A-5/1000147-2005-02-28.pdf',
 'input/X-17A-5/1000147-2006-03-02.pdf',
 'input/X-17A-5/1000147-2007-03-01.pdf',
 'input/X-17A-5/1000147-2008-02-29.pdf',
 'input/X-17A-5/1000147-2009-03-02.pdf',
 'input/X-17A-5/1000147-2010-03-01.pdf',
 'input/X-17A-5/1000147-2011-03-02.pdf',
 'input/X-17A-5/1000148-2002-03-01.pdf',
 'input/X-17A-5/1000148-2003-02-28.pdf',
 'input/X-17A-5/1000148-2004-03-01.pdf',
 'input/X-17A-5/1000148-2005-03-01.pdf',
 'input/X-17A-5/1000148-2006-02-24.pdf',
 'input/X-17A-5/1000148-2007-03-01.pdf',
 'input/X-17A-5/1000148-2008-02-29.pdf',
 'input/X-17A-5/1000148-2009-02-27.pdf',
 'input/X-17A-5/1000148-2010-03-02.pdf',
 'input/X-17A-5/1000148-2011-02-28.pdf

In [10]:
len(x17)

93599

## After step 3: total PDFs subseted

In [11]:
pdf_sub = s3_session.list_s3_files(s3_bucket,'temp/X-17A-5-PDF-SUBSETS/')

In [32]:
pdf_sub 

['temp/X-17A-5-PDF-SUBSETS/1000146-2002-03-13-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000146-2003-03-03-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000146-2004-03-08-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2002-02-28-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2003-02-28-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2004-02-25-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2004-03-29-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2005-02-28-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2006-03-02-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2007-03-01-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2008-02-29-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2009-03-02-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2010-03-01-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000147-2011-03-02-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000148-2002-03-01-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000148-2003-02-28-subset.pdf',
 'temp/X-17A-5-PDF-SUBSETS/1000148-2004-03-01-subset.pdf

In [12]:
len(pdf_sub)

93597

## After step 4: Textract

In [14]:
pdf_raw = s3_session.list_s3_files(s3_bucket,'temp/X-17A-5-PDF-RAW/')

In [31]:
pdf_raw

['temp/X-17A-5-PDF-RAW/1000146-2002-03-13.csv',
 'temp/X-17A-5-PDF-RAW/1000146-2003-03-03.csv',
 'temp/X-17A-5-PDF-RAW/1000146-2004-03-08.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2002-02-28.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2003-02-28.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2004-02-25.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2004-03-29.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2005-02-28.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2006-03-02.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2007-03-01.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2008-02-29.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2009-03-02.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2010-03-01.csv',
 'temp/X-17A-5-PDF-RAW/1000147-2011-03-02.csv',
 'temp/X-17A-5-PDF-RAW/1000148-2002-03-01.csv',
 'temp/X-17A-5-PDF-RAW/1000148-2003-02-28.csv',
 'temp/X-17A-5-PDF-RAW/1000148-2004-03-01.csv',
 'temp/X-17A-5-PDF-RAW/1000148-2005-03-01.csv',
 'temp/X-17A-5-PDF-RAW/1000148-2006-02-24.csv',
 'temp/X-17A-5-PDF-RAW/1000148-2007-03-01.csv',
 'temp/X-17A-5-PDF-RAW/1000148-2008-02-2

In [15]:
len(pdf_raw)

86409

## After step 5: PDF Clean

In [16]:
pdf_clean = s3_session.list_s3_files(s3_bucket,'temp/X-17A-5-CLEAN-PDFS/')

In [34]:
pdf_clean

['temp/X-17A-5-CLEAN-PDFS/1000146-2002-03-13.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000146-2003-03-03.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000146-2004-03-08.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2002-02-28.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2003-02-28.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2004-02-25.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2004-03-29.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2005-02-28.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2006-03-02.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2007-03-01.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2008-02-29.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2009-03-02.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2010-03-01.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000147-2011-03-02.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000148-2002-03-01.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000148-2003-02-28.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000148-2004-03-01.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000148-2005-03-01.csv',
 'temp/X-17A-5-CLEAN-PDFS/1000148-2006-02-24.csv',
 'temp/X-17A-5-CLEAN-PDFS/10001

In [17]:
len(pdf_clean)

86408

## After step 6: PDF Split

In [36]:
pdf_split_asset = s3_session.list_s3_files(s3_bucket,'temp/X-17A-5-SPLIT-PDFS/Assets')

In [37]:
pdf_split_asset

['temp/X-17A-5-SPLIT-PDFS/Assets/1000146-2002-03-13.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000146-2003-03-03.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000146-2004-03-08.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000147-2003-02-28.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000147-2005-02-28.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000147-2006-03-02.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000147-2007-03-01.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000147-2008-02-29.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000147-2009-03-02.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000147-2010-03-01.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000147-2011-03-02.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000148-2002-03-01.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000148-2003-02-28.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000148-2005-03-01.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000148-2006-02-24.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000148-2007-03-01.csv',
 'temp/X-17A-5-SPLIT-PDFS/Assets/1000148-2008-02-29.csv',
 'temp/X-17A-5

In [38]:
len(pdf_split_asset)

77846

In [41]:
pdf_split_liable = s3_session.list_s3_files(s3_bucket,'temp/X-17A-5-SPLIT-PDFS/Liability & Equity/')

In [42]:
len(pdf_split_liable)

77846

## After step 7: create unstructured

In [22]:
s3_pointer.download_file(s3_bucket, 'temp/ERROR-TEXTRACT.json', 'temp.json')        
# read data on errors derived from Textract
with open('temp.json', 'r') as f: error_dictionary = json.loads(f.read()) 
# remove local files for JSON
os.remove('temp.json')

In [23]:
error_dictionary == "'Blocks'"

False

In [24]:
len(error_dictionary)

3727

In [25]:
error_dictionary

{'1068940-2010-03-01': 'No Balance Sheet found, or parsing error',
 '1554888-2021-03-16': 'No Balance Sheet found, or parsing error',
 '1557131-2015-08-28': 'No Balance Sheet found, or parsing error',
 '1557131-2017-08-14': 'No Balance Sheet found, or parsing error',
 '1558345-2016-02-29': 'No Balance Sheet found, or parsing error',
 '1561014-2015-02-23': 'No Balance Sheet found, or parsing error',
 '1563580-2015-02-25': 'No Balance Sheet found, or parsing error',
 '1563961-2019-05-30': 'No Balance Sheet found, or parsing error',
 '1564449-2015-03-02': 'No Balance Sheet found, or parsing error',
 '1564449-2016-02-29': 'No Balance Sheet found, or parsing error',
 '1564449-2017-07-20': 'No Balance Sheet found, or parsing error',
 '1564449-2018-03-05': 'No Balance Sheet found, or parsing error',
 '1564449-2019-02-19': 'No Balance Sheet found, or parsing error',
 '1564449-2020-02-24': 'No Balance Sheet found, or parsing error',
 '1565138-2016-02-29': 'No Balance Sheet found, or parsing err

In [27]:
err = np.array(list(error_dictionary.values()))

In [28]:
sum(err == 'No Balance Sheet found, or parsing error')

3335

In [29]:
len(err[err != 'No Balance Sheet found, or parsing error'])

392

In [44]:
raw_pdf_files = s3_session.list_s3_files(s3_bucket, input_pdf)

parse_years = np.arange(1993,2022)

s3_pointer.download_file(s3_bucket, temp_folder + 'CIKandDealers.json', 'temp.json')
with open('temp.json', 'r') as f: old_cik2brokers = json.loads(f.read())
cik2brokers = dealerData(years=parse_years,company_email=company_email, cik2brokers=old_cik2brokers)   

# re-assign contents with new additional information 
os.remove('temp.json')

broker_dealers = cik2brokers['broker-dealers'].keys()


textract_files = list(filter(lambda x: brokerFilter(broker_dealers, x), raw_pdf_files))
from run_file_extraction import brokerFilter


EXTRACTING BROKER-DEALER INFORMATION
We have covered 1993/QTR1
We have covered 1993/QTR2
We have covered 1993/QTR3
We have covered 1993/QTR4
We have covered 1994/QTR1
We have covered 1994/QTR2
We have covered 1994/QTR3
We have covered 1994/QTR4
We have covered 1995/QTR1
We have covered 1995/QTR2
We have covered 1995/QTR3
We have covered 1995/QTR4
We have covered 1996/QTR1
We have covered 1996/QTR2
We have covered 1996/QTR3
We have covered 1996/QTR4
We have covered 1997/QTR1
We have covered 1997/QTR2
We have covered 1997/QTR3
We have covered 1997/QTR4
We have covered 1998/QTR1
We have covered 1998/QTR2
We have covered 1998/QTR3
We have covered 1998/QTR4
We have covered 1999/QTR1
We have covered 1999/QTR2
We have covered 1999/QTR3
We have covered 1999/QTR4
We have covered 2000/QTR1
We have covered 2000/QTR2
We have covered 2000/QTR3
We have covered 2000/QTR4
We have covered 2001/QTR1
We have covered 2001/QTR2
We have covered 2001/QTR3
We have covered 2001/QTR4
We have covered 2002/QTR1
W

In [46]:
error_dictionary

{'1068940-2010-03-01': 'No Balance Sheet found, or parsing error',
 '1554888-2021-03-16': 'No Balance Sheet found, or parsing error',
 '1557131-2015-08-28': 'No Balance Sheet found, or parsing error',
 '1557131-2017-08-14': 'No Balance Sheet found, or parsing error',
 '1558345-2016-02-29': 'No Balance Sheet found, or parsing error',
 '1561014-2015-02-23': 'No Balance Sheet found, or parsing error',
 '1563580-2015-02-25': 'No Balance Sheet found, or parsing error',
 '1563961-2019-05-30': 'No Balance Sheet found, or parsing error',
 '1564449-2015-03-02': 'No Balance Sheet found, or parsing error',
 '1564449-2016-02-29': 'No Balance Sheet found, or parsing error',
 '1564449-2017-07-20': 'No Balance Sheet found, or parsing error',
 '1564449-2018-03-05': 'No Balance Sheet found, or parsing error',
 '1564449-2019-02-19': 'No Balance Sheet found, or parsing error',
 '1564449-2020-02-24': 'No Balance Sheet found, or parsing error',
 '1565138-2016-02-29': 'No Balance Sheet found, or parsing err

In [61]:
for counter, pdf_paths in enumerate(textract_files):
    basefile = pdf_paths.split('/')[-1].split('-subset')[0]
    try:
        if error_dictionary[basefile] == "'Blocks'":
            print(counter)
        else:
            continue
    except:
        continue
    print('yipa')

48692
yipa
48693
yipa
48698
yipa
48790
yipa
49906
yipa
50307
yipa
50709
yipa
51299
yipa
51300
yipa
51301
yipa
51496
yipa
51497
yipa
51693
yipa
52092
yipa
52169
yipa
52303
yipa
52393
yipa
52396
yipa
52397
yipa
52399
yipa
52400
yipa
52696
yipa
52701
yipa
52798
yipa
52799
yipa
52896
yipa
52999
yipa
53001
yipa
53090
yipa
53193
yipa
53194
yipa
53195
yipa
53198
yipa
53199
yipa
53501
yipa
53502
yipa
53999
yipa
54101
yipa
54103
yipa
54499
yipa
54600
yipa
54698
yipa
54700
yipa
54794
yipa
54796
yipa
54895
yipa
54897
yipa
54899
yipa
55298
yipa
55299
yipa
55300
yipa
55697
yipa
55699
yipa
55706
yipa
55892
yipa
55893
yipa
55895
yipa
55896
yipa
55897
yipa
55898
yipa
55996
yipa
55999
yipa
56000
yipa
56002
yipa
56098
yipa
56100
yipa
56197
yipa
56492
yipa
56494
yipa
56496
yipa
56809
yipa
57296
yipa
57299
yipa
57300
yipa
57699
yipa
57899
yipa
58016
yipa
58017
yipa
58018
yipa
58190
yipa
58699
yipa
58700
yipa
58990
yipa
59091
yipa
59092
yipa
59094
yipa
59096
yipa
59097
yipa
59195
yipa
59199
yipa
59398
yipa